# Comparison of Full Grid Stereo with Window-based and Scanline Approaches

## Team members and contributions

- Soyeon Shin:

- Umer Amjad:

## Code Libraries
(acknowledge all external code libraries "essential" for your project. You should explain the importance of each such library for your project. If these libraries are not installed with the default anaconda package, provide installation instructions/links or include the necessary files in extlibs subdirectory.)

## Abstract
two-three paragraph abstract outlining the high-level goal and discussing well-motivated methodology of the project

## Window-based Stereo

Description TODO

In [6]:
%matplotlib inline
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt
import matplotlib.image as image
from matplotlib.colors import LogNorm
from skimage import img_as_ubyte
from skimage.color import rgb2grey

INFTY = np.inf
def SD_array(imageL, imageR, d_minimum, d_maximum):
    # initialization of the array of "squared differences" for different shifts
    SD = np.zeros((1+d_maximum-d_minimum,np.shape(imageL)[0],np.shape(imageL)[1]))
    imageL = imageL.astype(np.float)
    imageR = imageR.astype(np.float)
    for d in range(1 + d_maximum - d_minimum):
        delta = d_minimum + d
        dR = (imageL[:,:,0] - np.roll(imageR[:,:,0], delta, axis=1)).flatten()
        dG = (imageL[:,:,1] - np.roll(imageR[:,:,1], delta, axis=1)).flatten()
        dB = (imageL[:,:,2] - np.roll(imageR[:,:,2], delta, axis=1)).flatten()
        D = np.vstack((dR, dG, dB))
        SD[d] = (la.norm(D, axis=0) ** 2).reshape((np.shape(imageL)[0], np.shape(imageL)[1]))
    return SD

def integral_image(img):
    int_img = np.zeros(np.shape(img))
    for row in range(np.shape(img)[0]):
        row_sum = 0.
        for col in range(np.shape(img)[1]):
            row_sum += img[row][col]
            int_img[row][col] = row_sum
            if row > 0:
                int_img[row][col] += int_img[row-1][col]
    return int_img

def windSum(img, window_width):
    if window_width == 1:
        return img
    # position of point relative to bottom/right
    p = int(window_width//2)
    # partial sums with zero-filled margins
    BR = integral_image(img)
    BL = np.roll(BR, window_width, axis=1)
    BL[:,:window_width] = 0
    TR = np.roll(BR, window_width, axis=0)
    TR[:window_width,:] = 0
    TL = np.roll(np.roll(BR, window_width, axis=0), window_width, axis=1)
    TL[:,:window_width] = 0
    TL[:window_width,:] = 0
    # calculate window sum
    window_sum = BR - BL - TR + TL
    # fill bottom/right margins with INFTY
    window_sum[:p,:] = INFTY
    window_sum[:,:p] = INFTY
    # rotate sum to top/left position
    window_sum = np.roll(np.roll(window_sum, -p, axis=1), -p, axis=0)
    # fill top/left margins with INFTY
    q = p - 1 if (window_width % 2 == 0) else p
    window_sum[:,:q] = INFTY
    window_sum[:q,:] = INFTY
    return window_sum

def SSDtoDmap(SSD_array, d_minimum, d_maximum):
    return d_minimum + np.argmin(SSD_array, axis=0)

def Dmap_Windows(imageL, imageR, d_minimum, d_maximum, window_width):
    SD = SD_array(imageL, imageR, d_minimum, d_maximum)
    SSD = np.zeros(np.shape(SD))
    for d in range(1 + d_maximum - d_minimum):
        SSD[d] = windSum(SD[d], window_width)
    dMap = SSDtoDmap(SSD, d_minimum, d_maximum)
    return dMap

im_left = image.imread("images/Bicycle1-imperfect/im0.png")
im_right = image.imread("images/Bicycle1-imperfect/im1.png")
# im_gt = image.imread("images/Bicycle1-imperfect/disp0.pfm")
d_min = np.amin(im_gt[im_gt != 0]) / 16
d_max = np.amax(im_gt[im_gt != 0]) / 16
dispMap_small = Dmap_Windows(im_left, im_right, d_min, d_max, 4)
dispMap_large = Dmap_Windows(im_left, im_right, d_min, d_max, 15)
fig = plt.figure(figsize = (16, 7))
plt.subplot(131)
plt.title("disparity map (small windows)")
plt.imshow(dispMap_small, vmin = 0, vmax = d_max)
plt.subplot(132)
plt.title("disparity map (large windows)")
plt.imshow(dispMap_large, vmin = 0, vmax = d_max)
plt.subplot(133)
plt.title("ground truth disparity map ")
plt.imshow(im_gt/16, vmin = 0, vmax = d_max)
plt.colorbar(cax=plt.axes([0.91, 0.3, 0.01, 0.4]))

OSError: cannot identify image file 'images/Bicycle1-imperfect/disp0.pfm'

## Scan-line Stereo

## Full Grid Stereo

## Conclusion
two-three paragraphs summarizing the conclusions of your project